In [35]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
import torch
from tqdm import tqdm

Processing category: Tone:  67%|██████▋   | 4/6 [04:44<02:22, 71.21s/it]


In [2]:
input_file = "part-00000-b2e6f621-8b04-46d9-8d3f-cdc2903b682f-c000.snappy.parquet"
df = pd.read_parquet("/content/part-00000-b2e6f621-8b04-46d9-8d3f-cdc2903b682f-c000.snappy.parquet")
df

,parent_asin,title,description,reviews
0,0007161069,A Sense of the World: How a Blind Man Became H...,"[From Publishers Weekly, Starred Review., In t...",[This is one of the best books you could hope ...
1,0008116644,The Tolkien Treasury,"[Review, Praise for books in this treasury:""To...",[Outstanding! The stories are classic and the ...
2,0008276250,The Wives: Emily Charlton is Back in a New Dev...,[],[This book is the exact content as when life g...
3,0028625846,The History Atlas of Europe (History Atlas Ser...,"[Amazon.com Review, Europe's internal boundari...",[The graphics (and overall production values) ...
4,0028633784,The Complete Idiot's Guide to American Literature,"[From the Back Cover, You're no idiot, of cour...","[Based on recommendations here, I bought this ..."
...,...,...,...,...
108998,B09DDR1HPD,Grocery Row Gardening: The Exciting New Permac...,[],[I've utilized the system that David talks abo...
108999,B09N8M6LBN,One of Us Is Dead,"[About the Author, JENEVA ROSE, is the bestsel...",[Ok first off this is the second book by Jenev...
109000,B09NRDSMK4,Water Bath Canning And Preserving Cookbook For...,[],[The book claims to be over 200 WATER BATH CAN...
109001,B09QNWXQBV,The Words: An Enemies-to-Lovers Standalone Rom...,[],[Where do I even start with this book? Stella...


In [3]:
def get_book_by_title(book_title):

    if not book_title.strip():
        print("Error: The title provided is empty.")
        return None

    result = df[df['title'].str.contains(book_title, case=False, na=False)]

    if result.empty:
        print("No entries found with the specified title.")
        return None
    else:
        return result

In [22]:
get_book_by_title("All Quiet on the Western Front")

,parent_asin,title,description,reviews,text_input
56856,B00DAD25O8,All Quiet on the Western Front: A Novel,"[Review, “The world has a great writer in Eric...",[There are books you read in your youth deemed...,All Quiet on the Western Front: A Novel. Descr...
66334,B0045JKECG,All Quiet on the Western Front: NOW AN OSCAR A...,[],[Truly sad story. At first a bit graphic but ...,All Quiet on the Western Front: NOW AN OSCAR A...
68439,0449911497,All Quiet on the Western Front,"[Review, “The world has a great writer in Eric...",[This book is labeled as a novel but is actual...,All Quiet on the Western Front. Description: R...
78464,1441482652,All Quiet On The Western Front,[],"[This a very important work, perhaps a little ...",All Quiet On The Western Front. Description: ....
103334,0449213943,All Quiet on the Western Front: A Novel,"[Review, “The world has a great writer in Eric...",[What a great work of fiction. Feeling ashame...,All Quiet on the Western Front: A Novel. Descr...


0441172717
0545139708
B00760BPQA
0316219185
0345391802
B00DAD25O8

In [3]:
tags = {
    "Genres and Subgenres": [
        "Fantasy", "Science Fiction", "Mystery & Thriller", "Historical Fiction",
        "Romantic Fiction", "Literary Fiction", "Young Adult", "Children's Books",
        "Horror", "Dystopian", "Crime Fiction", "Non-fiction", "Memoir & Biography",
        "Self-Help", "Adventure", "Graphic Novels & Comics"
    ],
    "Themes": [
        "Coming of Age", "Family Drama", "War & Conflict", "Forbidden Love",
        "Identity & Self-Discovery", "Cultural Heritage", "Morality & Ethics",
        "Justice & Revenge", "Existentialism", "Friendship & Loyalty", "Loss & Grief",
        "Good vs. Evil", "Survival", "Politics & Power", "Technology & Futurism",
        "Mental Health"
    ],
    "Narrative Styles": [
        "Multiple POV", "Unreliable Narrator", "First Person Perspective",
        "Third Person Omniscient", "Non-linear Timeline", "Epistolary",
        "Stream of Consciousness", "Vivid Descriptions", "Character-Driven",
        "Plot-Twists", "Flashbacks"
    ],
    "Audience": [
        "For Fans of Classic Literature", "For Romance Enthusiasts", "For Thrill Seekers",
        "For Sci-Fi Geeks", "For History Buffs", "For Fantasy Lovers",
        "For Casual Readers", "For Young Readers", "For Philosophical Minds"
    ],
    "Tone": [
        "Dark & Gritty", "Uplifting & Inspirational", "Heartwarming", "Suspenseful",
        "Satirical", "Thought-Provoking", "Whimsical", "Humorous", "Bittersweet",
        "Fast-Paced", "Slow-Burning"
    ],
    "Setting": [
        "Medieval", "Post-Apocalyptic", "Urban Fantasy", "Alternate History",
        "Rural Towns", "Space Opera", "Victorian Era", "Middle Ages", "Modern-Day",
        "Future Earth", "Parallel Universes", "Mythical Worlds", "Island Adventure",
        "Futuristic Cityscapes"
    ],
    "Special Features": [
        "Series Starter", "Standalone", "Debut Novel", "Award-Winning", "Banned Books",
        "Plot-Driven", "Character Study", "Short Stories", "Long-Running Series",
        "Cult Classic", "Hidden Gem", "Retelling", "Audiobook Version", "Movie Adaptation"
    ]
}

In [4]:
book_codes = ["0441172717", "0545139708", "B00760BPQA", "0316219185", "0345391802", "B00DAD25O8"]


selected_books_df = df[df['parent_asin'].isin(book_codes)]


selected_books_df[['parent_asin', 'title']]


,parent_asin,title
15839,0545139708,Harry Potter and the Deathly Hallows (Harry Po...
41186,0441172717,Dune
52387,0345391802,The Hitchhiker's Guide to the Galaxy
56856,B00DAD25O8,All Quiet on the Western Front: A Novel
58425,B00760BPQA,A Dance With Dragons: Part 2 After The Feast: ...
93677,0316219185,"Baptism of Fire (The Witcher, 5)"


In [5]:
selected_books_df['text_input'] = selected_books_df.apply(
    lambda row: f"{row['title']}. Description: {' '.join(row['description'])}. Reviews: {' '.join(row['reviews'])}",
    axis=1
)
selected_books_df

<ipython-input-5-86981b34a19c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_books_df['text_input'] = selected_books_df.apply(


,parent_asin,title,description,reviews,text_input
15839,0545139708,Harry Potter and the Deathly Hallows (Harry Po...,"[About the Author, J.K. ROWLING is the author ...",[I ordered this book for my daughter and she c...,Harry Potter and the Deathly Hallows (Harry Po...
41186,0441172717,Dune,"[Review, Praise for, Dune, “I know nothing com...",[Frank Herbert built three dynasties inside a ...,Dune. Description: Review Praise for Dune “I k...
52387,0345391802,The Hitchhiker's Guide to the Galaxy,"[Amazon.com Review, Join Douglas Adams's haple...",[I remember listening to sporadic episodes of ...,The Hitchhiker's Guide to the Galaxy. Descript...
56856,B00DAD25O8,All Quiet on the Western Front: A Novel,"[Review, “The world has a great writer in Eric...",[There are books you read in your youth deemed...,All Quiet on the Western Front: A Novel. Descr...
58425,B00760BPQA,A Dance With Dragons: Part 2 After The Feast: ...,"[About the Author, George R.R. Martin is the a...","[Fans, beware. This is not new material. It's ...",A Dance With Dragons: Part 2 After The Feast: ...
93677,0316219185,"Baptism of Fire (The Witcher, 5)","[Review, ""This is a series you can sink your t...",[I absolutely love whole series of The Witcher...,"Baptism of Fire (The Witcher, 5). Description:..."


In [25]:
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [30]:
def classify_with_t5(text, tags):
    """
    Classifies the given text into the most relevant tags using T5.
    """
    print(text)
    prompt = (
        f"Identify relevant tags from the following list: {', '.join(tags)}. "
        f"Respond only with the relevant tags separated by commas. Book description: {text}"
    )
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=50)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Generated Text: {generated_text}")

    predicted_tags = [tag.strip() for tag in generated_text.split(',') if tag.strip() in tags]
    return predicted_tags


In [32]:
for category, category_tags in tags.items():
    selected_books_df[category] = [
        classify_with_t5(text, category_tags)
        for text in tqdm(selected_books_df['text_input'], desc=f"Processing category: {category}")
    ]



Processing category: Genres and Subgenres:   0%|          | 0/6 [00:00<?, ?it/s]

Harry Potter and the Deathly Hallows (Harry Potter, Book 7): Volume 7. Description: About the Author J.K. ROWLING is the author of the enduringly popular, era-defining Harry Potter seven-book series, which have sold over 600 million copies in 85 languages, been listened to as audiobooks for over one billion hours and made into eight smash hit movies. To accompany the series, she wrote three short companion volumes for charity, including Fantastic Beasts and Where to Find Them , which went on to inspire a new series of films featuring Magizoologist Newt Scamander. Harry’s story as a grown-up was continued in a stage play, Harry Potter and the Cursed Child , which J.K. Rowling wrote with playwright Jack Thorne and director John Tiffany. In 2020, she returned to publishing for younger children with the fairy tale The Ickabog , the royalties for which she donated to her charitable trust, Volant, to help charities working to alleviate the social effects of the Covid 19 pandemic. Her latest 



Processing category: Genres and Subgenres:  17%|█▋        | 1/6 [00:00<00:02,  1.99it/s]

Generated Text: the book. I ordered this book for my daughter and it jumped to 620. I missed the return window. I ordered this book for my daughter and she couldn’t put it down. I’ve missed the return window.
Dune. Description: Review Praise for Dune “I know nothing comparable to it except The Lord of the Rings .”—Arthur C. Clarke“It is possible that Dune is even more relevant now than when it was first published.”— The New Yorker “An astonishing science fiction phenomenon.”— The Washington Post “One of the monuments of modern science fiction.”— Chicago Tribune “Powerful, convincing, and most ingenious.”—Robert A. Heinlein   “Herbert’s creation of this universe, with its intricate development and analysis of ecology, religion, politics and philosophy, remains one of the supreme and seminal achievements in science fiction.”— Louisville Times From the Back Cover Here is the novel that will be forever considered a triumph of the imagination. Set on the desert planet Arrakis, Dune is the s



Processing category: Genres and Subgenres:  33%|███▎      | 2/6 [00:01<00:02,  1.54it/s]

...

Generated Text: : the author. the author. commas. from the following list: Series Starter, Standalone, Debut Novel, Award-Winning, Banned Books, Hidden Gem, Hidden Gem,
A Dance With Dragons: Part 2 After The Feast: The bestselling classic epic fantasy series behind the award-winning HBO and Sky TV show and phenomenon GAME OF THRONES (A Song of Ice and Fire, Book 5). Description: About the Author George R.R. Martin is the author of fourteen novels, including five volumes of A SONG OF ICE AND FIRE, several collections of short stories and numerous screen plays for television drama and feature films. He lives in Santa Fe, New Mexico. --This text refers to an out of print or unavailable edition of this title. Review `Colossal, staggering... Martin captures all the intoxicating complexity of the Wars of the Roses or Imperial Rome in his imaginary world ... one of the greats of fantasy literature'SFX`This is one of those rare and effortless reads' ROBIN HOBB`George R.R. Martin is one of our 



Processing category: Special Features:  83%|████████▎ | 5/6 [00:02<00:00,  2.06it/s]

Generated Text: - -,,,,,: The classic epic fantasy series behind the award-winning HBO and Sky TV show and phenomenon GAME OF THRONES (A Song of Ice and Fire
Baptism of Fire (The Witcher, 5). Description: Review "This is a series you can sink your teeth into."― BuzzFeed News "Delightful, intense, irreverent, and compelling....you have to read The Witcher books because they are rife with all of the elements that make you love fiction, and especially fantasy, in the first place....In a word, The Witcher delivers."― Hypable "One of the best and most interesting fantasy series I've ever read."― Nerds of a Feather "Like Mieville and Gaiman, [Sapkowski] takes the old and makes it new ... fresh take on genre fantasy."― Foundation "Sapkowski has a confident and rich voice which permeates the prose and remains post-translation. I'd recommend this to any fan of heroic or dark fiction."― SF Book Reviews About the Author Andrzej Sapkowski is the author of the Witcher series and the Hussite Trilogy



Processing category: Special Features: 100%|██████████| 6/6 [00:02<00:00,  2.05it/s]

Generated Text: :: of Fire,,,,: The Witcher, 5. the best. You have to read it. You have to read it. You have to read it. You have to read it


In [33]:
output_df_sample = selected_books_df[['parent_asin', 'title'] + list(tags.keys())]
output_df_sample

,parent_asin,title,Genres and Subgenres,Themes,Narrative Styles,Audience,Tone,Setting,Special Features
15839,0545139708,Harry Potter and the Deathly Hallows (Harry Po...,[],"[Family Drama, War & Conflict]","[Unreliable Narrator, First Person Perspective...",[],"[Uplifting & Inspirational, Heartwarming, Susp...","[Post-Apocalyptic, Urban Fantasy, Alternate Hi...",[]
41186,0441172717,Dune,[],[],[],[],[],"[Urban Fantasy, Alternate History, Rural Towns...",[]
52387,0345391802,The Hitchhiker's Guide to the Galaxy,"[Science Fiction, Mystery & Thriller, Historic...","[Family Drama, War & Conflict, Forbidden Love,...","[Unreliable Narrator, First Person Perspective...","[For Sci-Fi Geeks, For History Buffs, For Hist...","[Uplifting & Inspirational, Heartwarming, Susp...","[Alternate History, Rural Towns, Space Opera, ...",[]
56856,B00DAD25O8,All Quiet on the Western Front: A Novel,"[Science Fiction, Mystery & Thriller, Historic...","[Family Drama, War & Conflict, Forbidden Love,...","[Unreliable Narrator, First Person Perspective...",[For Romance Enthusiasts],"[Uplifting & Inspirational, Heartwarming, Susp...","[Post-Apocalyptic, Urban Fantasy, Alternate Hi...","[Standalone, Debut Novel, Award-Winning, Banne..."
58425,B00760BPQA,A Dance With Dragons: Part 2 After The Feast: ...,"[Fantasy, Science Fiction, Mystery & Thriller,...",[],[],[],"[Uplifting & Inspirational, Heartwarming, Susp...",[],[]
93677,0316219185,"Baptism of Fire (The Witcher, 5)","[Science Fiction, Mystery & Thriller, Historic...","[Family Drama, War & Conflict]","[Unreliable Narrator, First Person Perspective...",[For Romance Enthusiasts],"[Uplifting & Inspirational, Heartwarming, Susp...","[Post-Apocalyptic, Urban Fantasy, Alternate Hi...",[]


In [34]:
selected_books_df = df[df['parent_asin'].isin(book_codes)]
selected_books_df['text_input'] = selected_books_df.apply(
    lambda row: f"{row['title']}. Description: {' '.join(row['description'])}. Reviews: {' '.join(row['reviews'])}",
    axis=1
)

<ipython-input-34-cc56a7e47fc7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_books_df['text_input'] = selected_books_df.apply(


In [41]:
device = 0 if torch.cuda.is_available() else -1

In [42]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [43]:
def classify_with_zero_shot(text, tags):

    result = classifier(text, candidate_labels=tags, multi_label=True)
    predicted_tags = [label for label, score in zip(result['labels'], result['scores']) if score > 0.5]
    return predicted_tags

In [44]:
for category, category_tags in tags.items():
    selected_books_df[category] = [
        classify_with_zero_shot(text, category_tags)
        for text in tqdm(selected_books_df['text_input'], desc=f"Processing category: {category}")
    ]



Processing category: Genres and Subgenres:   0%|          | 0/6 [00:00<?, ?it/s]

Processing category: Genres and Subgenres:  17%|█▋        | 1/6 [00:03<00:15,  3.07s/it]

Processing category: Genres and Subgenres:  33%|███▎      | 2/6 [00:07<00:14,  3.64s/it]

Processing category: Genres and Subgenres:  50%|█████     | 3/6 [00:11<00:11,  3.84s/it]

Processing category: Genres and Subgenres:  67%|██████▋   | 4/6 [00:15<00:07,  3.94s/it]

Processing category: Genres and Subgenres:  83%|████████▎ | 5/6 [00:17<00:03,  3.22s/it]

Processing category: Genres and Subgenres: 100%|██████████| 6/6 [00:21<00:00,  3.61s/it]
<ipython-input-44-b5f065f35916>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_books_df[category] = [


Processing categor

In [45]:
output_df_sample = selected_books_df[['parent_asin', 'title'] + list(tags.keys())]
output_df_sample

,parent_asin,title,Genres and Subgenres,Themes,Narrative Styles,Audience,Tone,Setting,Special Features
15839,0545139708,Harry Potter and the Deathly Hallows (Harry Po...,"[Horror, Young Adult, Dystopian, Children's Bo...","[Mental Health, Identity & Self-Discovery, Goo...","[Non-linear Timeline, Multiple POV, First Pers...",[For Young Readers],"[Bittersweet, Fast-Paced, Suspenseful]",[Future Earth],"[Retelling, Movie Adaptation, Character Study,..."
41186,0441172717,Dune,"[Literary Fiction, Memoir & Biography, Histori...","[Existentialism, Identity & Self-Discovery, Fr...","[Unreliable Narrator, Stream of Consciousness,...","[For Romance Enthusiasts, For Fans of Classic ...","[Uplifting & Inspirational, Thought-Provoking,...","[Parallel Universes, Futuristic Cityscapes, Mo...","[Award-Winning, Debut Novel, Character Study, ..."
52387,0345391802,The Hitchhiker's Guide to the Galaxy,"[Self-Help, Literary Fiction, Adventure, Histo...","[Existentialism, Identity & Self-Discovery, Su...","[Multiple POV, Unreliable Narrator, Character-...","[For Fans of Classic Literature, For Romance E...","[Uplifting & Inspirational, Suspenseful, Thoug...","[Parallel Universes, Modern-Day, Alternate His...","[Award-Winning, Plot-Driven, Character Study, ..."
56856,B00DAD25O8,All Quiet on the Western Front: A Novel,"[Self-Help, Young Adult, Horror, Memoir & Biog...","[Identity & Self-Discovery, Existentialism, Lo...","[Multiple POV, Character-Driven, Unreliable Na...","[For Fans of Classic Literature, For Thrill Se...","[Uplifting & Inspirational, Thought-Provoking,...","[Parallel Universes, Alternate History, Post-A...","[Character Study, Award-Winning, Retelling, De..."
58425,B00760BPQA,A Dance With Dragons: Part 2 After The Feast: ...,[Fantasy],[Good vs. Evil],"[Multiple POV, Character-Driven, Non-linear Ti...",[For Fans of Classic Literature],"[Bittersweet, Suspenseful, Thought-Provoking]",[],"[Long-Running Series, Award-Winning]"
93677,0316219185,"Baptism of Fire (The Witcher, 5)","[Self-Help, Memoir & Biography, Literary Ficti...","[Existentialism, Identity & Self-Discovery, Te...","[Epistolary, Vivid Descriptions, Multiple POV,...","[For Fans of Classic Literature, For Thrill Se...","[Uplifting & Inspirational, Suspenseful, Thoug...","[Mythical Worlds, Futuristic Cityscapes, Post-...","[Award-Winning, Audiobook Version, Plot-Driven..."


Пока всё очень плохо. Будем пытаться использовать другие языковые модели. На примете Llama 3